<a href="https://colab.research.google.com/github/dpostolovski/eeg_is/blob/train_compare_full_data/Phase2_DataRead.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>
  <img alt="FINKI **LOGO**" height="30px" src="https://www.finki.ukim.mk/Content/dataImages/downloads/logo-large-500x500_2.png" hspace="10px" vspace="0px">
  Интелигентни системи - Лабораториска вежба 2 (Претпроцесирање)
</h1>
<center><h3><i>Група 5<i><h3></center>

---



In [0]:
#@title Монтирање на Google Drive податочниот систем
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#@title Инсталирање и вчитување на потребните библиотеки

# Библиотека за истражување, визуелизација и анализирање на човечки 
# неврофизиолошки податоци (EEG, sEEG и др)
!pip install mne 
!pip install termcolor

import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import pandas as pd
from termcolor import colored
import mne
from sklearn.decomposition import PCA, FastICA
import mne

In [0]:
%tensorflow_version 1.12.0

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.12.0`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
!wget "https://raw.githubusercontent.com/vlawhern/arl-eegmodels/master/EEGModels.py"
!mkdir saved_models

--2020-05-24 14:25:40--  https://raw.githubusercontent.com/vlawhern/arl-eegmodels/master/EEGModels.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18283 (18K) [text/plain]
Saving to: ‘EEGModels.py.1’

EEGModels.py.1      100%[===================>]  17.85K  --.-KB/s    in 0.001s  

2020-05-24 14:25:40 (22.0 MB/s) - ‘EEGModels.py.1’ saved [18283/18283]

mkdir: cannot create directory ‘saved_models’: File exists


In [0]:
!readlink -f ~/.keras/keras.json

/root/.keras/keras.json


In [0]:
#@title Fourier analysis
# Вчитување на податоците

from sklearn import metrics 
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint 
from sklearn.model_selection import train_test_split
from EEGModels import EEGNet,ShallowConvNet
import scipy.io as sio
from scipy.fft import fft
from scipy import signal
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras import backend as K
from tensorflow import keras
K.set_image_data_format('channels_first')

print(open("/root/.keras/keras.json").read())

# Вчитување на податоците
data = loadmat('drive/My Drive/Интелигентни Системи/Data/SBJ01/S01-Train/trainData.mat')['trainData'] 

for i in range(1, 2):
  file_name = 'SBJ' + format(i, '02')
  for j in range(1, 4):
    if i == 1 and j == 1: continue
    file_train_set = 'S' + format(j, '02') + '-Train'
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/trainData.mat"
    temp = loadmat(full_path)['trainData'] 
    data = np.concatenate((data, temp), axis=2)

print(data.size)
print(data.shape)

# Вчитување на label-ите
labels_arr = np.empty(0)
for i in range(1, 2):
  file_name = 'SBJ' + format(i, '02')
  for j in range(1, 4):
    file_train_set = 'S' + format(j, '02') + '-Train'
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      labels_arr = np.concatenate((labels_arr, temp))

print(labels_arr)
print(len(labels_arr))

# Вчитување на редоследот на светкање
events_arr = np.empty(0)
for i in range(1, 2):
  file_name = 'SBJ' + format(i, '02')
  for j in range(1, 4):
    file_train_set = 'S' + format(j, '02') + '-Train'
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      events_arr = np.concatenate((events_arr, temp))

# Вчитување на редоследот на објекти кои се target
targets_arr = np.empty(0)
for i in range(1, 2):
  file_name = 'SBJ' + format(i, '02')
  for j in range(1, 4):
    file_train_set = 'S' + format(j, '02') + '-Train'
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      targets_arr = np.concatenate((targets_arr, temp))



{
    "epsilon": 1e-07, 
    "floatx": "float32", 
    "image_data_format": "channels_last", 
    "backend": "tensorflow"
}
13440000
(8, 350, 4800)
['6' '6' '6' ... '3' '3' '3']
4800


In [0]:
data.shape


(8, 350, 4800)

In [0]:
#data = target_events_data_scaled
mne_array = np.swapaxes(data, 0, 2) # (епохa, канал, настан). 
mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)

In [0]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from EEGModels import EEGNet,ShallowConvNet,DeepConvNet
events_arr = events_arr.astype(np.int)
labels_arr = labels_arr.astype(np.int)
from tensorflow.keras.optimizers import Adam

#mne_array = data.reshape(2800, data.shape[-1])
#mne_array = np.swapaxes(mne_array,0,1)
X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


model = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
model.compile(loss = 'categorical_crossentropy', metrics=['Recall', 'Precision'],optimizer = Adam(0.0009))
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)


#clf = RandomForestClassifier(max_depth=5)
#clf.fit(X_train, y_train)
#score = clf.score(X_test, y_test)
#print(score)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_batch_size=100
num_epochs=400
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
          validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

score = model.evaluate(X_test, y_test, verbose=1)
print(score)

#model = Sequential()
#model.add(Dense(2800, input_dim=2800), activation='relu')
#model.add(Dense(350, activation='sigmoid'))
#model.add(Dense(350, activation='sigmoid'))
#model.add(Dense(1, activation='sigmoid'))

#clf = svm.SVC()
#clf = RandomForestClassifier(random_state=0, n_estimators=350)
#clf = LinearDiscriminantAnalysis()
#clf.fit(X_train, y_train)

Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3100/3600 [========================>.....] - ETA: 0s - loss: 2.2709 - recall_4: 0.0213 - precision: 0.2332
Epoch 00001: val_loss improved from inf to 2.18655, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 3s 845us/sample - loss: 2.2571 - recall_4: 0.0200 - precision: 0.2376 - val_loss: 2.1866 - val_recall_4: 0.0075 - val_precision: 0.4737
Epoch 2/400
3200/3600 [=========================>....] - ETA: 0s - loss: 2.1316 - recall_4: 0.0166 - precision: 0.3174
Epoch 00002: val_loss did not improve from 2.18655
3600/3600 [==============================] - 1s 148us/sample - loss: 2.1309 - recall_4: 0.0169 - precision: 0.3128 - val_loss: 2.1970 - val_recall_4: 0.0017 - val_precision: 0.2500
Epoch 3/400
3100/3600 [========================>.....] - ETA: 0s - loss: 2.1130 - recall_4: 0.0187 - precision: 0.3204
Epoch 00003: val_loss did not improve from 2.18655
3600/3600 [===========

In [0]:
print(y_test.shape)

(1200, 8)


In [0]:
labels_arr1 = labels_arr1.astype(np.int)
X_train, X_test, y_train, y_test = train_test_split(mne_array1, labels_arr1-1, test_size=0.99, random_state=42)
y_test = to_categorical(y_test)
score = model.evaluate(X_test, y_test, verbose=1)
print(X_test.shape)
print(y_test.shape)
print(score)

6336/6336 [==============================] - 1s 114us/sample - loss: 4.0363 - recall_4: 0.1703 - precision: 0.2270
(6336, 1, 8, 350)
(6336, 8)
[4.036261373096043, 0.17029671, 0.22701451]


In [0]:
# Вчитување на податоците

from sklearn import metrics 
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint 
from sklearn.model_selection import train_test_split
from EEGModels import EEGNet,ShallowConvNet
import scipy.io as sio
from scipy.fft import fft
from scipy import signal
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras import backend as K
from tensorflow import keras
K.set_image_data_format('channels_first')

print(open("/root/.keras/keras.json").read())

# Вчитување на податоците
data1 = loadmat('drive/My Drive/Интелигентни Системи/Test_Data/SBJ01/S04/Train/trainData.mat')['trainData'] 

for i in range(1, 2):
  file_name = 'SBJ' + format(i, '02')
  for j in range(4, 8):
    if i == 1 and j == 4: continue
    file_train_set = 'S' + format(j, '02')
    full_path = 'drive/My Drive/Интелигентни Системи/Test_Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData'] 
    data1 = np.concatenate((data1, temp), axis=2)

print(data1.size)
print(data1.shape)

# Вчитување на label-ите
labels_arr1 = np.empty(0)
for i in range(1, 2):
  file_name = 'SBJ' + format(i, '02')
  for j in range(4, 8):
    file_train_set = 'S' + format(j, '02') 
    full_path = 'drive/My Drive/Интелигентни Системи/Test_Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      labels_arr1 = np.concatenate((labels_arr, temp))

print(labels_arr1)
print(len(labels_arr1))

# Вчитување на редоследот на светкање
events_arr1 = np.empty(0)
for i in range(1, 2):
  file_name = 'SBJ' + format(i, '02')
  for j in range(4, 8):
    file_train_set = 'S' + format(j, '02') 
    full_path = 'drive/My Drive/Интелигентни Системи/Test_Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      events_arr1 = np.concatenate((events_arr1, temp))

# Вчитување на редоследот на објекти кои се target
targets_arr1 = np.empty(0)
for i in range(1, 2):
  file_name = 'SBJ' + format(i, '02')
  for j in range(4, 8):
    file_train_set = 'S' + format(j, '02') 
    full_path = 'drive/My Drive/Интелигентни Системи/Test_Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      targets_arr1 = np.concatenate((targets_arr1, temp))

{
    "epsilon": 1e-07, 
    "floatx": "float32", 
    "image_data_format": "channels_last", 
    "backend": "tensorflow"
}
17920000
(8, 350, 6400)
['6' '6' '6' ... '6' '6' '6']
6400


In [0]:
#data = target_events_data_scaled
mne_array1 = np.swapaxes(data1, 0, 2) # (епохa, канал, настан). 
mne_array1 = np.swapaxes(mne_array1, 1, 2) # (епохa, канал, настан). 
mne_array1 = mne_array1.reshape(mne_array1.shape[0],1, 8,350)

In [0]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from EEGModels import EEGNet,ShallowConvNet,DeepConvNet
events_arr1 = events_arr.astype(np.int)
labels_arr1 = labels_arr.astype(np.int)
from tensorflow.keras.optimizers import Adam

X_train, X_test, y_train, y_test = train_test_split(mne_array1, labels_arr1-1, test_size=0.25, random_state=42)


model1 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
model1.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)


y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_batch_size=100
num_epochs=400
model1.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
          validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)


score = model1.evaluate(X_test, y_test, verbose=1)
print(score)


ValueError: ignored